<a href="https://colab.research.google.com/github/LujainAK/GP/blob/main/LBPCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Feature Extraction Using LBP**

###Import Libraries

In [ ]:
pip install tensorflow-io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 81.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.35.0
    Uninstalling tensorflow-io-gcs-filesystem-0.35.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.35.0


In [ ]:
from google.colab import drive
import cv2
import tensorflow_io as tfio
from skimage import feature
import os
import pandas as pd

###Create folders for LBP

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Paths for the important folders
ODIR_path = '/content/drive/MyDrive/Colab Notebooks/ODIR Dataset'
training_folder = '/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/training'
testing_folder = '/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/testing'
augmented_training = '/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/augmented_training'
augmented_testing = '/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/augmented_testing'

image_folders = [training_folder, testing_folder, augmented_training, augmented_testing]

In [ ]:
# Create general folders for LBP
LBP_path = os.path.join(ODIR_path, 'LBP')
os.makedirs(LBP_path, exist_ok=True)

LBP_training = os.path.join(LBP_path, 'LBP_training')
os.makedirs(LBP_training, exist_ok=True)

LBP_testing = os.path.join(LBP_path, 'LBP_testing')
os.makedirs(LBP_testing, exist_ok=True)

# Create the folders where Gray, Cb, and Cr images will be stored
# For training
lbpGrayPath_training = os.path.join(LBP_training, 'lbpGrayImages_training')
lbp_CbPath_training = os.path.join(LBP_training, 'lbpCbImages_training')
lbp_CrPath_training = os.path.join(LBP_training, 'lbpCrImages_training')

os.makedirs(lbpGrayPath_training, exist_ok=True)
os.makedirs(lbp_CbPath_training, exist_ok=True)
os.makedirs(lbp_CrPath_training, exist_ok=True)

# For testing
lbpGrayPath_testing = os.path.join(LBP_testing, 'lbpGrayImages_testing')
lbp_CbPath_testing = os.path.join(LBP_testing, 'lbpCbImages_testing')
lbp_CrPath_testing = os.path.join(LBP_testing, 'lbpCrImages_testing')

os.makedirs(lbpGrayPath_testing, exist_ok=True)
os.makedirs(lbp_CbPath_testing, exist_ok=True)
os.makedirs(lbp_CrPath_testing, exist_ok=True)

print("The folders for LBP have been created.")

The folders for LBP have been created.


In [ ]:
# Lists to combine the names of the images in the original images folders and the augmented images folders
training_images = set()
testing_images = set()

for folder in image_folders:
    for disease in os.listdir(folder):
      disease_folder = os.path.join(folder, disease)
      if folder in [training_folder, augmented_training]:
        training_images.update(os.listdir(disease_folder))
      elif folder in [testing_folder, augmented_testing]:
        testing_images.update(os.listdir(disease_folder))

In [ ]:
''' Function to load the images' paths into a list and return it '''
def load_images():
  dataset = []
  for folder in image_folders:
    for disease in os.listdir(folder):
      disease_folder = os.path.join(folder, disease)
      for image_file in os.listdir(disease_folder):
        image_path = os.path.join(disease_folder, image_file)
        dataset.append(image_path)

  print("All the images have been loaded.")
  return dataset

In [ ]:
# Load the images paths into the dataset list
dataset = load_images()

All the images have been loaded.


In [ ]:
# Check the dataset
dataset[:3]

['/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/training/Normal_training/2839_left.jpg',
 '/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/training/Normal_training/2955_left.jpg',
 '/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/training/Normal_training/2903_left.jpg']

In [ ]:
# Read and extract the features from each image, then store them in the corresponding folder
for image_path in dataset:
  print(image_path)
  img = cv2.imread(image_path)
  imgray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  ycbcr = tfio.experimental.color.rgb_to_ycbcr(img)
  y, cb, cr = ycbcr[:,:,0], ycbcr[:,:,1], ycbcr[:,:,2]

  lbp_grayIm = feature.local_binary_pattern(imgray, 8, 1)
  lbp_cbIm = feature.local_binary_pattern(cb, 8, 1)
  lbp_crIm = feature.local_binary_pattern(cr, 8, 1)

  # Write result out in the desired folder
  img_name = image_path.rsplit('/', 1)[-1]

  # Define the variables for storing the LBP images
  image_gray = ""
  image_Cb = ""
  image_Cr = ""

  # if the image is a training image, store it in the training folders
  if img_name in training_images:
    image_gray = os.path.join(lbpGrayPath_training, img_name)
    image_Cb = os.path.join(lbp_CbPath_training, img_name)
    image_Cr = os.path.join(lbp_CrPath_training, img_name)

  # If the image is a testing image, store it in the testing folders
  if img_name in testing_images:
    image_gray = os.path.join(lbpGrayPath_testing, img_name)
    image_Cb = os.path.join(lbp_CbPath_testing, img_name)
    image_Cr = os.path.join(lbp_CrPath_testing, img_name)

  cv2.imwrite(image_gray, lbp_grayIm)
  cv2.imwrite(image_Cb, lbp_cbIm)
  cv2.imwrite(image_Cr, lbp_crIm)

print("All the images have been feature extracted successfully.")

Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/augmented_training/Cataract_augmented_training/augmented_Cataract__138_8005901.jpg
/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/augmented_training/Cataract_augmented_training/augmented_Cataract__139_5562133.jpg
/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/augmented_training/Cataract_augmented_training/augmented_Cataract__140_9474829.jpg
/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/augmented_training/Cataract_augmented_training/augmented_Cataract__141_6332501.jpg
/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/augmented_training/Cataract_augmented_training/augmented_Cataract__142_4867722.jpg
/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/augmented_training/Cataract_augmented_training/augmented_Cataract__143_8157429.jpg
/content/drive/MyDrive/Colab Notebooks/ODIR Dataset/augmented_training/Cataract_augmented_training/augmented_Cataract__144_5822646.jpg
/con

In [ ]:
len(os.listdir(lbpGrayPath_training))

10055

In [ ]:
len(os.listdir(lbpGrayPath_testing))

4310